In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

/Users/a/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/a/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
PATH = '/Users/a/Desktop/lab/city24/city24_export.csv'


In [3]:
df = pd.read_csv(PATH, ',')

In [4]:
def remove_last(string):
    if ':' in string:
        return string[:-1]
    return string

def replace_space(string):
    return string.replace(' ', '_')
df.columns = df.columns.map(str.lower).map(remove_last).map(replace_space)
cols_to_remove = ['first_image_link', 'description', 'otseviide', 
                  'lisainfo', 'kinnisvara_tüüp', 'lisaväärtused', 'ventilatsioon']
df.drop(cols_to_remove, axis=1, inplace=True)


In [5]:
df.drop(index=df[df['address'].isnull()].index, inplace=True)
df.dropna(subset=['hind','üldpind', 'tubade_arv'], inplace=True)
df.drop(['hind/ühik'], axis=1, inplace=True)
df.drop(['broker_name', 'lift'], axis=1, inplace=True)
df.dropna(subset=['broker_company'], inplace=True)


    


In [6]:
# divide address
df_address = pd.DataFrame(df.address.str.split(',',1).tolist(),columns = ['address','city_part'])    
df_address.count()
display(df_address['address'])

0                         Ao tn 2 - 34
1                              Kuninga
2                             Merirahu
3                       Ringi tn 60-26
4              Toom-Kuninga tn 21 - 34
5                       Taludevahe 116
6                              Rahumäe
7                Telliskivi tn 49 - 67
8                Telliskivi tn 49 - 28
9                           Ravi tn 14
10               Telliskivi tn 49 - 75
11                     Linnamäe tee 38
12                     Lutheri kvartal
13                            Lasnamäe
14                        Jakobi tn 15
15                Väike-Sepa tn 11 - 1
16                     Sõpruse pst 244
17                Telliskivi tn 49 - 5
18                     Linnamäe tee 12
19                    Pelguranna tn 23
20                       HEA PAKKUMINE
21                          Ümera tn 2
22                              Soprdi
23                               Ahtme
24       MÖBLEERITUD KORTER HEAS KOHAS
25                     Li

In [7]:
#divide linnaosad
kesklinn = df.loc[df['address'].str.contains('Kesklinna linnaosa')]
haabersti = df.loc[df['address'].str.contains('Haabersti linnaosa')]
kristiine = df.loc[df['address'].str.contains('Kristiine linnaosa')]
nomme = df.loc[df['address'].str.contains('Nõmme linnaosa')]
vanalinn = df.loc[df['address'].str.contains('Vanalinn')]
mustamae = df.loc[df['address'].str.contains('Mustamäe linnaosa')]
lasnamae = df.loc[df['address'].str.contains('Lasnamäe linnaosa')]
pirita = df.loc[df['address'].str.contains('Pirita linnaosa')]
pohja_tallinn = df.loc[df['address'].str.contains('Põhja-Tallinna linnaosa')]
kadriorg = df.loc[df['address'].str.contains('Kadriorg')]

frames = [kesklinn, haabersti, kristiine, nomme, vanalinn, mustamae, lasnamae, pirita, pohja_tallinn, kadriorg]
tln_df = pd.concat(frames)
tln_df.shape


(2185, 16)

In [8]:
#divide address column
def get_linnaosa(address):
    tln_parts = ['Kesklinn', 
                 'Haabersti', 
                 'Kristiine', 
                 'Nõmme', 
                 'Vanalinn', 
                 'Mustamäe', 
                 'Lasnamäe', 
                 'Pirita', 
                 'Põhja-Tallinn', 
                 'Kadriorg']
    for part in address:
        for word in tln_parts:
            if word in part:
                return word
            else:
                'Tallinn'
                            
tln_df['city_part'] = tln_df.address.str.split(',').apply(lambda x: get_linnaosa(x))

In [9]:
tln_df.shape

(2185, 17)

In [10]:
import re
pattern = re.compile("^[A-z-. ÄÖÜÕäöüõ]+\s+\d+")
def get_address(address):
    for part in address:
        part = part.rstrip().lstrip()
        match_obj = pattern.match(part)
        if match_obj:
            return match_obj.group()
    return np.nan

tln_df['address'] = tln_df.address.str.split(',').apply(lambda x: get_address(x))
tln_df.dropna(subset=['address'], inplace=True)


In [11]:
tln_df.count()


address              2148
hind                 2148
üldpind              2148
ehitusaasta          1763
hoone_materjal       1777
rõdu                 1217
küttesüsteem         1929
energiamärgis        1819
seisukord            1929
tubade_arv           2148
magamistubade_arv    1778
vannitubade_arv      1718
korrus/korruseid     2135
parkimine            1084
sanitaar             1737
broker_company       2148
city_part            2148
dtype: int64

In [12]:
# hind column cleaning from EUR
def clean_numeric_value(row):
    row = row.strip(' ')
    numeric = re.sub(r'\D', "", row)
    return numeric
tln_df['hind'] = tln_df['hind'].apply(lambda x: clean_numeric_value(x))

# change type of hind column
tln_df['hind'] = tln_df['hind'].astype(int)
tln_df.dtypes

address               object
hind                   int64
üldpind               object
ehitusaasta          float64
hoone_materjal        object
rõdu                  object
küttesüsteem          object
energiamärgis         object
seisukord             object
tubade_arv           float64
magamistubade_arv    float64
vannitubade_arv      float64
korrus/korruseid      object
parkimine             object
sanitaar              object
broker_company        object
city_part             object
dtype: object

In [13]:
#clean uldpind from m and convert to float
def clean_floating_value(row):
    row = row.strip(' ')
    floating = re.sub(r'[+-]?([0-9]*[.])?[0-9]+', "", row)
    return floating

tln_df['üldpind'] = tln_df['üldpind'].str.split(' ').apply(lambda x: x[0])
tln_df['üldpind'] = tln_df['üldpind'].astype(float)


In [14]:
# clean ehitusaasta column
tln_df.loc[tln_df['address'].str.contains('Pärnu mnt 69'),'ehitusaasta'] = 1911

In [15]:
tln_df[tln_df['ehitusaasta'].isnull()].shape

(366, 17)

In [16]:
# find same house objects that have ehitusaasta column aggregate the major value and assign
null_year_address_list = tln_df[tln_df['ehitusaasta'].isnull()]['address'].tolist()
for address in null_year_address_list:
    year_isnotna = tln_df['ehitusaasta'].notnull()
    has_addr = tln_df['address'] == address
    found_year = tln_df[year_isnotna & has_addr]
    if not found_year.empty:
        found = found_year['ehitusaasta'].value_counts().index[0]
        tln_df.loc[tln_df['address'] == address, 'ehitusaasta'] = found
tln_df[tln_df['ehitusaasta'].isnull()].shape


(276, 17)

In [17]:
# assign average ehitusaasta column value to nans
tallinn_average_build_year = tln_df[tln_df['ehitusaasta'].notnull()]['ehitusaasta'].mean()
display(tallinn_average_build_year)
tln_df.loc[tln_df['ehitusaasta'].isnull(), 'ehitusaasta'] = tallinn_average_build_year
tln_df[tln_df['ehitusaasta'].isnull()].shape

#change ehitusaasta column dtype
tln_df['ehitusaasta'] = tln_df['ehitusaasta'].astype(int)


1993.5106837606838

In [29]:
# drop hoone_material column because a lot of missing values
tln_df.drop(['hoone_materjal'], axis=1, inplace=True)
display(tln_df)

,address,hind,üldpind,ehitusaasta,rõdu,küttesüsteem,energiamärgis,seisukord,tubade_arv,magamistubade_arv,vannitubade_arv,korrus/korruseid,parkimine,sanitaar,broker_company,city_part
0,Ao tn 2,649800,148.3,2018,"klaasitud rõdu, rõdu, terrass","keskküte, põrandaküte",b,uusehitis,4.0,3.0,3.0,5 / 5,maaalune parkla,"saun, vann, wc ja vannituba eraldi, dušš",Perton Ehitus OÜ,Kesklinn
10,Ravi tn 14,190000,154.2,2004,prantsuse rõdu,elektriküte,d,remonti vajav,5.0,4.0,2.0,4 / 5,maaalune parkla,"saun, vann, tsentraalne kanalisatsioon, wc ja ...",DE Kodu OÜ,Kesklinn
13,Pärnu mnt 69,75000,14.9,1911,NaN,keskküte,b,renoveeritud,1.0,1.0,1.0,2 / 3,hoovis,"tsentraalne kanalisatsioon, dušš",Fisk OÜ,Kesklinn
15,Jakobi tn 15,77900,36.9,1960,NaN,keskküte,NaN,"renoveeritud, uus viimistlus, valmis",1.0,1.0,1.0,2 / 4,"tasuta parkimine, parkimiskoht kinnistul","tsentraalne kanalisatsioon, dušš",City Property OÜ,Kesklinn
33,Narva mnt 9,165000,76.0,1958,NaN,keskküte,Energiamärgis puudub,"renoveeritud, valmis",3.0,3.0,1.0,5 / 7,NaN,"vann, tsentraalne kanalisatsioon, wc ja vannit...",Eesti Kinnisvarabüroo OÜ,Kesklinn
35,Tartu mnt 52,209900,79.5,2017,klaasitud rõdu,põrandaküte,b,uusehitis,3.0,2.0,1.0,5 / 5,maaalune parkla,"vann, wc ja vannituba eraldi, dušš",Merko Ehitus AS,Kesklinn
38,Tina 13,125000,64.6,1963,rõdu,keskküte,d,san remont tehtud,3.0,2.0,1.0,1 / 5,NaN,NaN,RE Kinnisvara AS,Kesklinn
40,Pärnu mnt 69,119000,39.2,1911,NaN,keskküte,b,renoveeritud,2.0,1.0,1.0,2 / 3,hoovis,"tsentraalne kanalisatsioon, dušš",Fisk OÜ,Kesklinn
51,Vesivärava tn 50,269500,105.4,2019,rõdu,"keskküte, põrandaküte",b,uusehitis,4.0,3.0,2.0,9 / 6,NaN,"vann, wc ja vannituba eraldi",Baltic Sotheby's International Realty - Estonia,Kesklinn
52,Rävala pst 11,65000,25.9,1951,NaN,keskküte,d,NaN,1.0,1.0,1.0,1 / 5,NaN,"vann, pesumasin",RE Kinnisvara AS,Kesklinn


(299, 17)

,address,hind,üldpind,ehitusaasta,hoone_materjal,rõdu,küttesüsteem,energiamärgis,seisukord,tubade_arv,magamistubade_arv,vannitubade_arv,korrus/korruseid,parkimine,sanitaar,broker_company,city_part
0,Ao tn 2,649800,148.3,2018,kivimaja,"klaasitud rõdu, rõdu, terrass","keskküte, põrandaküte",b,uusehitis,4.0,3.0,3.0,5 / 5,maaalune parkla,"saun, vann, wc ja vannituba eraldi, dušš",Perton Ehitus OÜ,Kesklinn
10,Ravi tn 14,190000,154.2,2004,paneelmaja,prantsuse rõdu,elektriküte,d,remonti vajav,5.0,4.0,2.0,4 / 5,maaalune parkla,"saun, vann, tsentraalne kanalisatsioon, wc ja ...",DE Kodu OÜ,Kesklinn
13,Pärnu mnt 69,75000,14.9,1911,kivimaja,NaN,keskküte,b,renoveeritud,1.0,1.0,1.0,2 / 3,hoovis,"tsentraalne kanalisatsioon, dušš",Fisk OÜ,Kesklinn
15,Jakobi tn 15,77900,36.9,1960,kivimaja,NaN,keskküte,NaN,"renoveeritud, uus viimistlus, valmis",1.0,1.0,1.0,2 / 4,"tasuta parkimine, parkimiskoht kinnistul","tsentraalne kanalisatsioon, dušš",City Property OÜ,Kesklinn
33,Narva mnt 9,165000,76.0,1958,kivimaja,NaN,keskküte,Energiamärgis puudub,"renoveeritud, valmis",3.0,3.0,1.0,5 / 7,NaN,"vann, tsentraalne kanalisatsioon, wc ja vannit...",Eesti Kinnisvarabüroo OÜ,Kesklinn
35,Tartu mnt 52,209900,79.5,2017,paneelmaja,klaasitud rõdu,põrandaküte,b,uusehitis,3.0,2.0,1.0,5 / 5,maaalune parkla,"vann, wc ja vannituba eraldi, dušš",Merko Ehitus AS,Kesklinn
38,Tina 13,125000,64.6,1963,kivimaja,rõdu,keskküte,d,san remont tehtud,3.0,2.0,1.0,1 / 5,NaN,NaN,RE Kinnisvara AS,Kesklinn
40,Pärnu mnt 69,119000,39.2,1911,kivimaja,NaN,keskküte,b,renoveeritud,2.0,1.0,1.0,2 / 3,hoovis,"tsentraalne kanalisatsioon, dušš",Fisk OÜ,Kesklinn
51,Vesivärava tn 50,269500,105.4,2019,NaN,rõdu,"keskküte, põrandaküte",b,uusehitis,4.0,3.0,2.0,9 / 6,NaN,"vann, wc ja vannituba eraldi",Baltic Sotheby's International Realty - Estonia,Kesklinn
52,Rävala pst 11,65000,25.9,1951,kivimaja,NaN,keskküte,d,NaN,1.0,1.0,1.0,1 / 5,NaN,"vann, pesumasin",RE Kinnisvara AS,Kesklinn
